# Introduction to CARTOframes
In this notebook, we will walk through the basics of CARTOframes including:

* Authenticating to your CARTO account
* Reading in a dataset from an external source
* Writing a dataset to your CARTO account
* Visualizing a dataset in your notebook
* Publishing a map and sharing a link 


## Installing CARTOframes
Let's first install CARTOframes (1.0 beta 2 version) and geopandas in this Google Colaboratory notebook. Note that the `bang` character (!) in front of the pip command allows us to run system commands on our virtual machine in Colaboratory from this notebook:

In [0]:
!pip install cartoframes==1.0b2 geopandas

     |████████████████████████████████| 133kB 4.8MB/s 
     |████████████████████████████████| 901kB 34.9MB/s 
     |████████████████████████████████| 51kB 23.6MB/s 
     |████████████████████████████████| 245kB 44.7MB/s 
     |████████████████████████████████| 9.8MB 53.0MB/s 
     |████████████████████████████████| 17.9MB 48.4MB/s 
  Created wheel for carto: filename=carto-1.6.0-cp36-none-any.whl size=30409 sha256=8e12dfb87efad134ea44cdb62fa21ff82839d787269950dca5fec7c9eab123fb
  Stored in directory: /root/.cache/pip/wheels/57/f0/8a/c177062c9ac07f5418e9e5d2a126008a37df9de9f1bfa6f7f9
  Created wheel for pyrestcli: filename=pyrestcli-0.6.12-cp36-none-any.whl size=8740 sha256=96eb4b39bcac3067449789e25a563f190879b101c6f1ec8de31cca986c3605e7
  Stored in directory: /root/.cache/pip/wheels/a9/d1/95/1f9b119bce04c308c8b1abae557d40df6ea0ab269e2ec12060
  Created wheel for munch: filename=munch-2.3.2-py2.py3-none-any.whl size=6614 sha256=13fd09c43869e74c4661707ce7ab8d0d6f74394b891abee041669445633

## Import libraries
Next, let's import libraries that we intend on using in this notebook:

In [0]:
#import libraries
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.viz import Map, Layer, Popup
from cartoframes.data import Dataset
from cartoframes.viz.helpers import color_continuous_layer
import pandas as pd
import geopandas as gpd

## Authenticating to your CARTO account
Now let's set up two types of authentication to our CARTO account. One will use `set_default credentials` to set authentication globaly across the notebook, and the second will use a `Credentials` class to allow us to pass credentials to functions for things like uploading a dataset to CARTO.

You will need two pieces of information from your account to authenticate. First is you CARTO username, second is an API Key with access the appropriate access to the tables in your account you would like to modify. You can find your username and API Keys at a URL of the form:

https://{{YOUR_USER_NAME}}.carto.com/your_apps


In [0]:
#@title Your credentials
USER_NAME = 'jdgodchaux-carto' #@param {type:"string"}
API_KEY = 'd22bf323314a8ff732fc71d4731984c8659112d8' #@param {type:"string"}


set_default_credentials(
    username=USER_NAME,
    api_key=API_KEY
)

credentials = Credentials(USER_NAME, API_KEY)


## Reading in a dataset from an external source
Now let's load some data from an open data source into a Pandas DataFrame. We'll use data on where food trucks are located, and what they're selling and their permit status. 

In [0]:
food_trucks_df = pd.read_csv('https://data.sfgov.org/resource/rqzj-sfat.csv')
food_trucks_df.head()

,objectid,applicant,facilitytype,cnn,locationdescription,address,blocklot,block,lot,permit,status,fooditems,x,y,latitude,longitude,schedule,dayshours,noisent,approved,received,priorpermit,expirationdate,location
0,1351263,Isidoro Serrano,Push Cart,1108000,21ST ST: CAPP ST to MISSION ST (3150 - 3199),2501 MISSION ST,3615070,3615,070,19MFF-00075,REQUESTED,NaN,6.007060e+06,2.103743e+06,37.756877,-122.418580,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,NaN,2019-07-12,0,2020-07-15T00:00:00.000,POINT (-122.418579889476 37.7568774515357)
1,1334734,Rita's Catering,Truck,9100000,MISSION ST: 06TH ST to 07TH ST (1000 - 1099),1028 MISSION ST,3703033,3703,033,19MFF-00047,APPROVED,Filipino Food,6.009812e+06,2.112361e+06,37.780694,-122.409669,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,2019-07-03T00:00:00.000,2019-07-03,0,2020-07-15T00:00:00.000,POINT (-122.409668813219 37.7806943774082)
2,1334819,tacos y pupusas los trinos,NaN,9158000,MISSION ST: AVALON AVE to COTTER ST (4368 - 4439),4384 MISSION ST,6798004,6798,004,19MFF-00048,REQUESTED,NaN,6.002680e+06,2.093159e+06,37.727567,-122.432970,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,NaN,2019-07-03,0,NaN,POINT (-122.432969701989 37.7275665375917)
3,1353225,Treats by the Bay LLC,Truck,7034000,HOWARD ST: MALDEN ALY to 02ND ST (574 - 599),201 02ND ST,3736097,3736,097,19MFF-00111,REQUESTED,Prepackaged Kettlecorn,6.013266e+06,2.114515e+06,37.786802,-122.397872,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,NaN,2019-07-31,0,2020-07-15T00:00:00.000,POINT (-122.397871635003 37.7868016505971)
4,1336685,Mike's Catering,Truck,7200000,INDIANA ST: 26TH ST to CESAR CHAVEZ ST (1500 -...,1575 INDIANA ST,4317017,4317,017,19MFF-00054,APPROVED,Cold Truck: packaged sandwiches: snacks: candy...,6.015268e+06,2.101381e+06,37.750850,-122.390028,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,2019-07-10T00:00:00.000,2019-07-10,1,2020-07-15T00:00:00.000,POINT (-122.390027615501 37.7508496068125)


Now, let's see what fields are in this food truck DataFrame:

In [0]:
food_trucks_df.columns

Index(['objectid', 'applicant', 'facilitytype', 'cnn', 'locationdescription',
       'address', 'blocklot', 'block', 'lot', 'permit', 'status', 'fooditems',
       'x', 'y', 'latitude', 'longitude', 'schedule', 'dayshours', 'noisent',
       'approved', 'received', 'priorpermit', 'expirationdate', 'location'],
      dtype='object')

## Visualizing a dataset in your notebook
Notice that we have `Latitude` and `Longitude` fields with values. With latitudes and longitudes in hand we can make a map directly as we will do below. If latitude and longitude data we not avaialble, then we could leverage [CARTO's Data Services API](https://carto.com/developers/data-services-api/reference/#geocoding-functions) to geocode using street addresses. This will be a topic covered in more advanced CARTOframes education notebooks. 

Let's make a map of food trucks and push carts in San Francisco, making a GeoDataFrame first to ensure our latitude and longitudes are encoded as geometries, then using the `Dataset` class to allow the GeoDataFrame to be mapable with CARTOframes:

In [0]:
food_trucks_gdf = gpd.GeoDataFrame(food_trucks_df, geometry=gpd.points_from_xy(food_trucks_df.longitude, food_trucks_df.latitude))

food_trucks_dataset = Dataset(food_trucks_gdf)
Map(Layer(food_trucks_dataset))

Great, we have a map, but it also looks like one or more of the food truck locations is a latitude = 0 and longitude = 0 ([Null Island!](https://en.wikipedia.org/wiki/Null_Island)), so let's remove 0 values from the GeoDataFrame and then try making out map again:

In [0]:
food_trucks_gdf = food_trucks_gdf[food_trucks_gdf.latitude != 0]
food_trucks_gdf = food_trucks_gdf[food_trucks_gdf.longitude != 0]

# We could also remove null values like this:
food_trucks_gdf = food_trucks_gdf[food_trucks_gdf.latitude.notnull()]
food_trucks_gdf = food_trucks_gdf[food_trucks_gdf.longitude.notnull()]

food_trucks_dataset = Dataset(food_trucks_gdf)
Map(Layer(food_trucks_dataset))

Great! Now let's say we want to know where existing food trucks and carts are located by cuisine. Let's first filter our dataset to only see food trucks where `status` equals `APPROVED`:

In [0]:
approved_food_trucks_gdf = food_trucks_gdf[food_trucks_gdf.status == 'APPROVED']
approved_food_trucks_gdf.head()

,objectid,applicant,facilitytype,cnn,locationdescription,address,blocklot,block,lot,permit,status,fooditems,x,y,latitude,longitude,schedule,dayshours,noisent,approved,received,priorpermit,expirationdate,location,geometry
1,1334734,Rita's Catering,Truck,9100000,MISSION ST: 06TH ST to 07TH ST (1000 - 1099),1028 MISSION ST,3703033,3703,033,19MFF-00047,APPROVED,Filipino Food,6009812.085,2112361.070,37.780694,-122.409669,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,2019-07-03T00:00:00.000,2019-07-03,0,2020-07-15T00:00:00.000,POINT (-122.409668813219 37.7806943774082),POINT (-122.409668813219 37.7806943774082)
4,1336685,Mike's Catering,Truck,7200000,INDIANA ST: 26TH ST to CESAR CHAVEZ ST (1500 -...,1575 INDIANA ST,4317017,4317,017,19MFF-00054,APPROVED,Cold Truck: packaged sandwiches: snacks: candy...,6015267.752,2101380.964,37.750850,-122.390028,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,2019-07-10T00:00:00.000,2019-07-10,1,2020-07-15T00:00:00.000,POINT (-122.390027615501 37.7508496068125),POINT (-122.390027615501 37.7508496068125)
5,1339625,El Calamar Perubian Food Truck,Truck,9091000,MISSION ST: ANTHONY ST to 02ND ST (573 - 599),85 02ND ST,3708019,3708,019,19MFF-00099,APPROVED,Lomo Saltado: Jalea: Ceviche: Calamar: Tilapia...,6012696.635,2115129.488,37.788457,-122.399884,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,2019-07-22T00:00:00.000,2019-07-22,0,2020-07-15T00:00:00.000,POINT (-122.399884160566 37.7884570288289),POINT (-122.399884160566 37.78845702882889)
6,1328724,Linda's Catering,Truck,12662000,TOWNSEND ST: LUSK ST to 04TH ST (252 - 299),260 TOWNSEND ST,3787024,3787,024,19MFF-00021,APPROVED,Hot Dogs: Hamburgers: Nachos: Steaks: Pastas: ...,6014193.326,2111344.676,37.778148,-122.394441,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,2019-06-12T00:00:00.000,2019-06-12,1,2020-07-15T00:00:00.000,POINT (-122.394440688559 37.7781481274607),POINT (-122.394440688559 37.7781481274607)
7,1334953,DO UC US Mobile Catering,Truck,8700000,MARIN ST: KANSAS ST to HWY 101 N ON RAMP (2500...,2590 MARIN ST,4339008,4339,008,19MFF-00050,APPROVED,Cold truck: sandwiches: salads: beverages: chi...,6011441.697,2100557.577,37.748376,-122.403201,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,2019-07-05T00:00:00.000,2019-07-05,1,2020-07-15T00:00:00.000,POINT (-122.403200626496 37.7483758257779),POINT (-122.403200626496 37.7483758257779)


Now let's take a look at the universe of `fooditems` to see if this would be a reasonable variable to group by:

In [0]:
approved_food_trucks_gdf.fooditems.unique()

array(['Filipino Food',
       'Cold Truck: packaged sandwiches: snacks: candy: hot and cold drinks',
       'Lomo Saltado: Jalea: Ceviche: Calamar: Tilapia plate: chicken special. Soda: Water.',
       'Hot Dogs: Hamburgers: Nachos: Steaks: Pastas: Asian Dishes: Tri-Tip Sandwiches: Sodas & Water',
       'Cold truck: sandwiches: salads: beverages: chips: candy: cookies: coffee: tea: drinks',
       'Ice Cream: Waffle Cones', 'Tacos: burritos: quesadillas',
       'Cold Truck: Burrito: Corn Dog: Salads: Sandwiches: Quesadilla: Tacos: Fried Rice: Cow Mein: Chinese Rice: Noodle Plates: Soup: Bacon: Eggs: Ham: Avacado: Sausages: Beverages',
       'Cold Truck: Corn Dogs: Noodle Soups: Candy: Pre-packaged Snacks: Sandwiches: Chips: Coffee: Tea: Various Beverages',
       'Acai Bowls: Smoothies: Juices',
       'Burritos: Tacos: Tortas: Quesadillas & Various Drinks.',
       'Hot dogs: condiments: soft pretzels: soft drinks: coffee: cold beverages: pastries: bakery goods: cookies: ice cream

That's a lot of different unique values! Let's just use the first set of words before the first colon character, and then see how many unique values we see.

In [0]:
fooditems_group = approved_food_trucks_gdf.fooditems.str.split(":", n = 1, expand = True)
approved_food_trucks_gdf['fooditems_group'] = fooditems_group[0]
approved_food_trucks_gdf.head()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,objectid,applicant,facilitytype,cnn,locationdescription,address,blocklot,block,lot,permit,status,fooditems,x,y,latitude,longitude,schedule,dayshours,noisent,approved,received,priorpermit,expirationdate,location,geometry,fooditems_group
1,1334734,Rita's Catering,Truck,9100000,MISSION ST: 06TH ST to 07TH ST (1000 - 1099),1028 MISSION ST,3703033,3703,033,19MFF-00047,APPROVED,Filipino Food,6009812.085,2112361.070,37.780694,-122.409669,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,2019-07-03T00:00:00.000,2019-07-03,0,2020-07-15T00:00:00.000,POINT (-122.409668813219 37.7806943774082),POINT (-122.409668813219 37.7806943774082),Filipino Food
4,1336685,Mike's Catering,Truck,7200000,INDIANA ST: 26TH ST to CESAR CHAVEZ ST (1500 -...,1575 INDIANA ST,4317017,4317,017,19MFF-00054,APPROVED,Cold Truck: packaged sandwiches: snacks: candy...,6015267.752,2101380.964,37.750850,-122.390028,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,2019-07-10T00:00:00.000,2019-07-10,1,2020-07-15T00:00:00.000,POINT (-122.390027615501 37.7508496068125),POINT (-122.390027615501 37.7508496068125),Cold Truck
5,1339625,El Calamar Perubian Food Truck,Truck,9091000,MISSION ST: ANTHONY ST to 02ND ST (573 - 599),85 02ND ST,3708019,3708,019,19MFF-00099,APPROVED,Lomo Saltado: Jalea: Ceviche: Calamar: Tilapia...,6012696.635,2115129.488,37.788457,-122.399884,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,2019-07-22T00:00:00.000,2019-07-22,0,2020-07-15T00:00:00.000,POINT (-122.399884160566 37.7884570288289),POINT (-122.399884160566 37.78845702882889),Lomo Saltado
6,1328724,Linda's Catering,Truck,12662000,TOWNSEND ST: LUSK ST to 04TH ST (252 - 299),260 TOWNSEND ST,3787024,3787,024,19MFF-00021,APPROVED,Hot Dogs: Hamburgers: Nachos: Steaks: Pastas: ...,6014193.326,2111344.676,37.778148,-122.394441,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,2019-06-12T00:00:00.000,2019-06-12,1,2020-07-15T00:00:00.000,POINT (-122.394440688559 37.7781481274607),POINT (-122.394440688559 37.7781481274607),Hot Dogs
7,1334953,DO UC US Mobile Catering,Truck,8700000,MARIN ST: KANSAS ST to HWY 101 N ON RAMP (2500...,2590 MARIN ST,4339008,4339,008,19MFF-00050,APPROVED,Cold truck: sandwiches: salads: beverages: chi...,6011441.697,2100557.577,37.748376,-122.403201,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,2019-07-05T00:00:00.000,2019-07-05,1,2020-07-15T00:00:00.000,POINT (-122.403200626496 37.7483758257779),POINT (-122.403200626496 37.7483758257779),Cold truck


Now let's create a map categorizing food trucks by color:

In [0]:
approved_food_trucks_dataset = Dataset(approved_food_trucks_gdf)

approved_food_truck_map = Map(
    Layer(
        approved_food_trucks_dataset,
        '''
        color: ramp($fooditems_group, bold)
        width: 15
        ''',
        legend={
           'type': 'color-category',
           'title': 'Type of Food Sold'
        },
        popup={
            'hover': {
                'title': 'Name',
                'value': '$applicant'
            },
            'click': [{
                'title': 'Location',
                'value': '$locationdescription'
            },{
                'title': 'Food Sold',
                'value': '$fooditems'
            }]
        }
    )
)

approved_food_truck_map

## Writing a dataset to your CARTO account, publishing a map & sharing a link
Now that we have a filtered dataset just the way we would like it, let's upload that dataset to our CARTO account, publish that map and share that map with our colleages:

In [0]:
approved_food_truck_map.sync_data('approved_food_trucks_dataset')
approved_food_truck_map.publish('approved_food_truck_map', maps_api_key=API_KEY)

{'id': 'bd0051e2-6060-4b51-b2ba-7456ed4fd932',
 'name': 'approved_food_truck_map',
 'privacy': 'public',
 'url': 'https://team.carto.com/u/jdgodchaux-carto/kuviz/bd0051e2-6060-4b51-b2ba-7456ed4fd932'}

## Just writing a dataset to your CARTO account
If you only want to write your dataset to your CARTO account without publishing or sharing maps, then use the `upload` fucntion available in your dataset: 

In [0]:
approved_food_trucks_dataset.upload(table_name='approved_food_trucks_dataset', if_exists='replace', credentials=credentials)